In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.4 MB/s eta 0:00:00


In [3]:
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, AdamW, get_cosine_schedule_with_warmup
import pandas as pd
import numpy as np
import json
import math
import torch
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F

In [14]:
model_name = r"bert-base-uncased"
qasper_classification_path = r"/content/drive/MyDrive/Colab Notebooks/Qasper_classification.json"

device = r"cuda"
tokenizer = AutoTokenizer.from_pretrained(model_name)
attributes = [
                "unanswerable",
                "extractive_spans",
                "yes_no",
                "abstractive"
            ]
max_token_length = 512
bert_model = AutoModel.from_pretrained(model_name, return_dict = True)
batch_size = 8

n_labels = 4
lr = 1.5e-6
warmup = 0.2
weight_decay = 0.001
n_epochs = 8
MODEL_PATH = r"/content/drive/MyDrive/Colab Notebooks/model.bin"

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
class Qasper_Dataset(Dataset):

    def __init__(self, data_path, data_type, tokenizer, attributes, max_token_length = 128, sample = None):
        self.data_path = data_path
        self.data_type = data_type
        self.tokenizer = tokenizer
        self.attributes = attributes
        self.max_token_length = max_token_length
        self.data = self.__load_data__()

    def __load_data__(self):
        data_fd = open(self.data_path)
        self.data = json.load(data_fd)
        return (
            self.data['data'][self.data_type]
        )

    def __len__(self):
        return (
            len(self.data)
        )

    def __getitem__(self, index):
        context = ""
        for i in range(len(self.data[index][0])):
            if i != 0:
                context = context + " "
            context = context + self.data[index][0][i]
        # context = self.data[index][0]
        question = self.data[index][1]
        labels = [self.data[index][2][self.attributes[0]], self.data[index][2][self.attributes[1]], self.data[index][2][self.attributes[2]], self.data[index][2][self.attributes[3]]]
        labels = torch.FloatTensor(labels)
        # labels = Variable(labels, requires_grad = True)
        tokens = self.tokenizer.encode_plus(
                                            question,
                                            context,
                                            add_special_tokens=True,
                                            return_tensors='pt',
                                            truncation=True,
                                            padding='max_length',
                                            max_length=self.max_token_length,
                                            return_attention_mask = True
                                            )
        return {'input_ids': tokens.input_ids.flatten(), 'attention_mask': tokens.attention_mask.flatten(), 'labels': labels}

In [6]:
train_data = Qasper_Dataset(
                data_path = qasper_classification_path,
                data_type = "train_data",
                tokenizer = tokenizer,
                attributes = attributes,
                max_token_length = 512,
                sample = None
                )

val_data = Qasper_Dataset(
                    data_path = qasper_classification_path,
                    data_type = "validation_data",
                    tokenizer = tokenizer,
                    attributes = attributes,
                    max_token_length = 512,
                    sample = None
                    )

train_dataloader = DataLoader(
                  train_data,
                  batch_size = batch_size,
                  num_workers = 2,
                  shuffle = True
              )

val_dataloader = DataLoader(
                  val_data,
                  batch_size = batch_size,
                  num_workers = 2,
                  shuffle = True
              )

In [7]:
class Qasper_Classifier(nn.Module):

    def __init__(self, model, n_labels):

        super(Qasper_Classifier, self).__init__()
        self.pretrained_model = model # bert model
        self.n_labels = n_labels

        self.hidden = torch.nn.Linear(self.pretrained_model.config.hidden_size, self.pretrained_model.config.hidden_size)
        print(f"hidden - {self.hidden.weight.requires_grad}")
        self.classifier = torch.nn.Linear(self.pretrained_model.config.hidden_size, self.n_labels)
        print(f"classifier - {self.classifier.weight.requires_grad}")
        torch.nn.init.xavier_uniform_(self.classifier.weight)
        # print(f"loss - {self.loss_func.weight.requires_grad}")
        self.dropout = nn.Dropout(0.3)
        # print(f"dropout - {self.dropout.weight.requires_grad}")


    def forward(self, input_ids, attention_mask, labels = None):
        # print("Forward Propagation started")
        # input_ids.requires_grad = True; attention_mask,requires_grad = True
        print(f"input - {input_ids.requires_grad}, attention_mask - {attention_mask.requires_grad}")
        output = self.pretrained_model(
                                    input_ids = input_ids,
                                    attention_mask = attention_mask
                                       )
        pooled_output = torch.mean(output.last_hidden_state, 1)
        pooled_output = self.dropout(pooled_output)
        pooled_output = self.hidden(pooled_output)
        pooled_output = F.relu(pooled_output)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        # loss = 0
        # loss.requires_grad = True
        # logits.requires_grad = True
        # labels.required_grad = True
        # loss = Variable(loss, requires_grad = True)
        return logits

In [8]:
def train_fn(data_loader, model, optimizer, device, scheduler):

  model.train()

  for idx, data in tqdm(enumerate(data_loader), total = len(data_loader)):
    # print(data.keys())
    print(f"Training loop {idx}")
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    labels = data['labels']

    input_ids = input_ids.to(device, dtype=torch.long)
    attention_mask = attention_mask.to(device, dtype=torch.long)
    labels = labels.to(device, dtype=torch.float)

    optimizer.zero_grad()
    logits = model(
                  input_ids = input_ids,
                  attention_mask = attention_mask,
              )

    loss = torch.zeros(1, requires_grad=True)
    # if labels is not None:
    print(f"type - {type(logits)} type - {type(labels)}")
    loss_fn = nn.BCEWithLogitsLoss(reduction='mean')
    loss = loss_fn(logits.view(-1, len(attributes)), labels.view(-1, len(attributes)))
    print(f"loss - {loss}, logits - {logits}")
    print(f"loss__ - {loss.requires_grad}, logits__ - {logits.requires_grad}")

    loss.backward()
    optimizer.step()
    scheduler.step()

In [9]:
def eval_fn(data_loader, model):

  model.eval()

  tot_logits = []
  tot_labels = []

  with torch.no_grad():
    for idx, data in tqdm(enumerate(data_loader), total = len(data_loader)):
      print(f"Validation Loop {idx}")
      input_ids = data['input_ids']
      attention_mask = data['attention_mask']
      labels = data['labels']

      input_ids = input_ids.to(device, dtype=torch.long)
      attention_mask = attention_mask.to(device, dtype=torch.long)
      labels = labels.to(device, dtype=torch.float)

      logits = model(
                    input_ids = input_ids,
                    attention_mask = attention_mask,
                )
      logits = logits.cpu().detach().numpy().tolist()
      labels = labels.cpu().detach().numpy().tolist()

      tot_logits.extend(logits)
      tot_labels.extend(labels)

    return (
      tot_logits,
      tot_labels
   )

In [10]:
model = Qasper_Classifier(bert_model, len(attributes))
param_optimizer = list(model.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
  {
      "params": [
          p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
      ],
      "weight_decay": 0.001,
  },
  {
      "params": [
          p for n, p in param_optimizer if any(nd in n for nd in no_decay)
      ],
      "weight_decay": 0.0,
  },
]

num_train_steps = int(len(train_data) / batch_size * n_epochs)
optimizer = AdamW(optimizer_parameters, lr=3e-5)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)

hidden - True
classifier - True


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [11]:
def clipping_fn(logits, max_val, min_val):

  for i in range(len(logits)):
    for j in range(len(logits[i])):

        if logits[i][j] >= (max_val + min_val)/2:
          logits[i][j] = max_val
        else:
          logits[i][j] = min_val

  return (
      logits
  )

def metric_accuracy(logits, labels):
  confusion = [
      {
         'tp': 0, 'tn': 0, 'fp': 0, 'fn': 0,
      },
      {
         'tp': 0, 'tn': 0, 'fp': 0, 'fn': 0,
      },
      {
         'tp': 0, 'tn': 0, 'fp': 0, 'fn': 0,
      },
      {
         'tp': 0, 'tn': 0, 'fp': 0, 'fn': 0,
      },
  ]

  for i in range(len(logits)):
    for j in range(len(logits[i])):

      if logits[i][j] == 1 and labels[i][j] == 1:
          confusion[j]['tp'] += 1

      if logits[i][j] == 0 and labels[i][j] == 0:
          confusion[j]['tn'] += 1

      if logits[i][j] == 1 and labels[i][j] == 0:
          confusion[j]['fn'] += 1

      if logits[i][j] == 0 and labels[i][j] == 1:
          confusion[j]['fp'] += 1

  print(confusion)

  accuracy = list()

  for i in range(len(confusion)):
    accuracy.append((confusion[i]['tp'] + confusion[i]['tn'])/(confusion[i]['tp'] + confusion[i]['tn'] + confusion[i]['fp'] + confusion[i]['fn']))

  return (
      accuracy
  )


In [12]:
def __run__():
    best_accuracy = 0
    model.to(device)
    print(model)

    for epoch in range(n_epochs):
        train_fn(train_dataloader, model, optimizer, device, scheduler)
        logits, labels = eval_fn(val_dataloader, model)
        logits = clipping_fn(logits, 1.0, 0.0)
        accuracy = metric_accuracy(logits, labels)
        overall_accuracy = (accuracy[0] + accuracy[1] + accuracy[2] + accuracy[3])/len(accuracy)

        print(f"Detailed accuracy after {epoch} epoch:")
        print(f"unanswerable accuarcy: {accuracy[0]}")
        print(f"extractive accuarcy: {accuracy[1]}")
        print(f"yes_no accuarcy: {accuracy[2]}")
        print(f"abstractive accuarcy: {accuracy[3]}")
        print(f"Overall accuarcy: {overall_accuracy}")
        print(f"Best accuarcy: {best_accuracy}")

        if overall_accuracy > best_accuracy:
            torch.save(model.state_dict(), MODEL_PATH)
            best_accuracy = overall_accuracy
            print(best_accuracy)
            print("Model Updated")

In [15]:
__run__()

Qasper_Classifier(
  (pretrained_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

  0%|          | 0/289 [00:00<?, ?it/s]

Training loop 0
input - False, attention_mask - False
type - <class 'torch.Tensor'> type - <class 'torch.Tensor'>
loss - 0.7028533816337585, logits - tensor([[ 0.1042,  0.0097, -0.2244, -0.1596],
        [ 0.1670, -0.0519, -0.0425,  0.0588],
        [ 0.1172, -0.0451, -0.1672, -0.1193],
        [-0.0832,  0.1046,  0.0711, -0.1118],
        [-0.1584,  0.1696,  0.0891, -0.1099],
        [ 0.4863, -0.0772, -0.3482,  0.2355],
        [ 0.1764,  0.0063, -0.0162,  0.0363],
        [ 0.1982,  0.0634, -0.0465,  0.2797]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
loss__ - True, logits__ - True


  0%|          | 1/289 [00:01<09:13,  1.92s/it]

Training loop 1
input - False, attention_mask - False
type - <class 'torch.Tensor'> type - <class 'torch.Tensor'>
loss - 0.6853101253509521, logits - tensor([[-0.1424, -0.0101, -0.3388,  0.0840],
        [-0.0852, -0.1045, -0.0972,  0.1649],
        [-0.3189, -0.2214, -0.4059,  0.4939],
        [ 0.0824, -0.1066, -0.1465,  0.1066],
        [ 0.0138, -0.2534, -0.2397,  0.1971],
        [ 0.0809,  0.1549, -0.3624,  0.2311],
        [-0.3401,  0.2418,  0.1447, -0.1136],
        [ 0.0332, -0.1696, -0.1445, -0.0409]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
loss__ - True, logits__ - True


...

Validation Loop 192
input - False, attention_mask - False


 99%|█████████▉| 193/194 [00:48<00:00,  3.97it/s]

Validation Loop 193
input - False, attention_mask - False


100%|██████████| 194/194 [00:49<00:00,  3.95it/s]

[{'tp': 0, 'tn': 1552, 'fp': 0, 'fn': 0}, {'tp': 818, 'tn': 374, 'fp': 143, 'fn': 217}, {'tp': 155, 'tn': 1367, 'fp': 5, 'fn': 25}, {'tp': 183, 'tn': 1014, 'fp': 248, 'fn': 107}]
Detailed accuracy after 7 epoch:
unanswerable accuarcy: 1.0
extractive accuarcy: 0.7680412371134021
yes_no accuarcy: 0.9806701030927835
abstractive accuarcy: 0.7712628865979382
Overall accuarcy: 0.879993556701031
Best accuarcy: 0.899645618556701


In [29]:
context = "Hello my name name is Preetam."
question = "What is my name?"

print("Loading Model...")

classification_model = torch.load(MODEL_PATH)

Loading Model...


In [32]:
tokens = tokenizer.encode_plus(
                                            question,
                                            context,
                                            add_special_tokens=True,
                                            return_tensors='pt',
                                            truncation=True,
                                            padding='max_length',
                                            max_length=max_token_length,
                                            return_attention_mask = True
                                            )
data = {'input_ids': tokens.input_ids.flatten(), 'attention_mask': tokens.attention_mask.flatten()}

In [38]:
input_ids = data['input_ids']
attention_mask = data['attention_mask']

input_ids = input_ids.to(device, dtype=torch.long)
attention_mask = attention_mask.to(device, dtype=torch.long)

# logits = bert_model(
#               input_ids = input_ids,
#               attention_mask = attention_mask,
#           )